# LLM Fine Tuning -llama on AWS(High Memory required)[Didn't work on AWS]

# Installing and importing the libraries

In [7]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\jimmy\\AppData\\Roaming\\Python\\Python311\\site-packages\\~-kenizers\\tokenizers.cp311-win_amd64.pyd'
Check the permissions.



In [8]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install huggingface_hub

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install tqdm==4.64.0

In [5]:
!pip install xformers

In [6]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

ImportError: cannot import name 'top_k_top_p_filtering' from 'transformers' (C:\Users\jimmy\AppData\Roaming\Python\Python311\site-packages\transformers\__init__.py)

In [ ]:
# check whether GPU/CPU is vaialble
torch.cuda.is_available()

# Loading the model

In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="aboonaji/llama2finetune-v2"
).to("cpu")  # Ensure the model is loaded on the CPU

llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

# Loading the tokenizer

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

# Chat with the model without updates

In [ ]:
user_prompt = "Please tell me about Ascariasis"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 50)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

# Setting the training arguments

In [ ]:
#training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 100)
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 10)

# Creating the Supervised Fine-Tuning trainer

In [ ]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

## Step 6: Training the model

In [ ]:
llama_sft_trainer.train()

# Chatting with the new updated model

In [ ]:
user_prompt = "Please tell me about Ascariasis"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 50)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])